In [1]:
import Random
using Distributions
using Statistics
using DelimitedFiles
using Dates

In [2]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [3]:
function data_collection(initialstate::Vector{Int64},params::Vector{Float64},outcomes,storage,numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG,ntherm)
    endstate,steps,weightsum = direct_gillespie!(initialstate,params,outcomes,transcription_translation_rates!,storage,updatestorage!,rng,numsteps,maxsteps,ntherm)
    return storage,endstate,steps,weightsum
end

function reach_balance_test(initialstate,params,outcomes,numsteps,maxsteps,rng,flux_thres,cov_thres,tries,ntherm)
    statevector = initialstate
    trynum = 0
    thresholdmet = false
    totalsteps = 0
    calcstor = BalanceEqData_nofb()
    fluxcheck = [0,0]
    cbvals = [0,0]
    
    while (trynum < tries) && (!thresholdmet)
        calcstor,statevector,steps,lastsimtime = data_collection(statevector,params,outcomes,calcstor,numsteps,maxsteps,rng,ntherm)
        weightsum = calcstor.weightsum
        fluxcheck = [relative_error(calcstor.meanRpX,calcstor.meanRmX),relative_error(calcstor.meanRpY,calcstor.meanRmY)]
        if (fluxcheck[1] < flux_thres) &&  (fluxcheck[2] < flux_thres)
            cbxx_comp = cov_balance_comp(calcstor.CXRmX/weightsum,calcstor.CXRmX/weightsum,
            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/weightsum,calcstor.CXRpX/weightsum) 
            cbyy_comp = cov_balance_comp(calcstor.CYRmY/weightsum,calcstor.CYRmY/weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/weightsum,calcstor.CYRpY/weightsum) 
            cbyx_comp = cov_balance_comp(calcstor.CYRmX/weightsum,calcstor.CXRmY/weightsum,
                            0,calcstor.CYRpX/weightsum,calcstor.CXRpY/weightsum) 
            cbvals = [cbxx_comp,cbyy_comp,cbyx_comp]
            thresholdmet = all(cbvals .< cov_thres)               
        end
        trynum += 1
        totalsteps += steps
        if maxsteps>100000
            maxsteps = maxsteps ÷ 2
        end
    end
return calcstor,totalsteps,trynum,fluxcheck,cbvals
end

reach_balance_test (generic function with 1 method)

Run some number of simulations, storing the used parameters+simulation constraints. Save each simulation's output into a big CSV. Finally, go through the results and calculate the perturbation data

In [4]:
is = [10,10]
params = [50.,1.,50.,5.]
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e6)
rng = Random.MersenneTwister()
fluxt = 0.01
covt = 0.01
tries = 20
ntherm = Int(10e3)
reach_balance_test(is,params,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)

(BalanceEqData_nofb(4065.5310581863796, 49.85423048964811, 12451.135606566042, 201338.74012747017, 2.1104133028388782e9, 7.786215266427637e6, 50.0, 49.8541379543915, 2492.706897719199, 2490.2278518127187, 201298.16410622053, 0.0, 7.787989330322846e6, 0.0, 1.5569075530544638e6, 1.0064908205312025e7, 4.2208310467179775e8, 3.893994665162058e8), 20859395, 20, [0.0029257761059261153, 0.000995509669798145], [0.008355195251035449, 0.053436159641028, 0.07704861069312277])

folder = "no_fb_sims_largerange4_"*Dates.format(today(),"yyyy-mm-dd")*"/"
mkdir(folder)
nsims = 80
is = [10,10]
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e8)
fluxt = 0.02
covt = 0.05
tries = 40
ntherm = Int(10e3)

paramsets = (
[50.,10.,5.,1.],
[40.,5.,40.,1.],
[10.,1.,15.,15.],
[25.,1.,2.,1.,])
for (k,params) in enumerate(paramsets)
    i = k
    println("Paramset $(paramsets)")
    pertparams = copy(params)
    open(folder*"paramset_$(i)_settings.tsv", "w") do io
        titles = ["params","outcomes","initialstate","numsteps","maxsteps","flux thres","cov thres","max tries","thermalisation"]
        data = [params,outcomes,is,numsteps,maxsteps,fluxt,covt,tries,ntherm]
        writedlm(io,reshape(titles,1,length(titles)))
        writedlm(io,reshape(data,1,length(data)))
    end;
    open(folder*"paramset_$(i)_simresults.tsv", "w") do io
        titles = ["sim #","Perturbed param","Pert Size","time","steps","flux thres","cov thres","<x>","<y>","Var(x)","Var(y)","cov(x,y)"]
        writedlm(io, reshape(titles,1,length(titles)))
    end;
    for j = 1:nsims
        pertrng = Random.Xoshiro()
        statevector = copy(is)
        pertsize = rand(pertrng,Uniform(0.8,1.2))
        pertparams[j%4+1] *= pertsize
        rng = Random.Xoshiro()
        calcstor,totalsteps,trynum,fluxcheck,cbvals = reach_balance_test(statevector,pertparams,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)
        open(folder*"paramset_$(i)_simresults.tsv", "a+") do io
            data = [j,j%4+1,pertsize,calcstor.weightsum,totalsteps,fluxcheck,cbvals,calcstor.meanx,calcstor.meany,calcstor.VX/calcstor.weightsum,calcstor.VY/calcstor.weightsum,calcstor.CXY/calcstor.weightsum]
            writedlm(io, reshape(data,1,length(data)))
        end;
        pertparams[j%4+1] /= pertsize
    end
end


Above I sample from an independent dist, but sample dist may not actually be ind? I think case below should force that

folder = "no_fb_sims_inddist_manysims_"*Dates.format(today(),"yyyy-mm-dd")*"/"
mkdir(folder)
nsims = 64 #Must be multiple of four to ensure independence I think
is = [10,10]
perts = [0.8,1.2]
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e8)
fluxt = 0.02
covt = 0.05
tries = 40
ntherm = Int(10e3)

paramsets = (
    [50.,10.,5.,1.],
    [40.,5.,40.,1.],
    [10.,1.,15.,15.],
    [25.,1.,2.,1.,])
for (k,params) in enumerate(paramsets)
    i = k
    println("Paramset $(paramsets)")
    pertparams = copy(params)
    open(folder*"paramset_$(i)_settings.tsv", "w") do io
        titles = ["params","outcomes","initialstate","numsteps","maxsteps","flux thres","cov thres","max tries","thermalisation"]
        data = [params,outcomes,is,numsteps,maxsteps,fluxt,covt,tries,ntherm]
        writedlm(io,reshape(titles,1,length(titles)))
        writedlm(io,reshape(data,1,length(data)))
    end;
    open(folder*"paramset_$(i)_simresults.tsv", "w") do io
        titles = ["sim #","Perturbed param","Pert Size","time","steps","flux thres","cov thres","<x>","<y>","Var(x)","Var(y)","cov(x,y)","usedparams"]
        writedlm(io, reshape(titles,1,length(titles)))
    end;
    for j = 1:nsims
        statevector = copy(is)
        pertsize = rand(perts)
        pertparams[2*(j%2)+1] *= pertsize
        rng = Random.Xoshiro()
        calcstor,totalsteps,trynum,fluxcheck,cbvals = reach_balance_test(statevector,pertparams,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)
        open(folder*"paramset_$(i)_simresults.tsv", "a+") do io
            data = [j,2*(j%2)+1,pertsize,calcstor.weightsum,totalsteps,fluxcheck,cbvals,calcstor.meanx,calcstor.meany,calcstor.VX/calcstor.weightsum,calcstor.VY/calcstor.weightsum,calcstor.CXY/calcstor.weightsum,pertparams]
            writedlm(io, reshape(data,1,length(data)))
        end;
        pertparams[2*(j%2)+1] /= pertsize
    end
end

folder = "no_fb_sims_stderrtest2_"*Dates.format(today(),"yyyy-mm-dd")*"/"
mkdir(folder)
nsims = 40 #Must be multiple of four to ensure independence I think
is = [10,10]
perts = [0.8,1.2]
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e8)
fluxt = 0.02
covt = 0.05
tries = 40
ntherm = Int(10e3)

paramsets = repeat([[25.,1.,5.,1.]],10)
for (k,params) in enumerate(paramsets)
    i = k
    println("Paramset $(paramsets)")
    pertparams = copy(params)
    open(folder*"paramset_$(i)_settings.tsv", "w") do io
        titles = ["params","outcomes","initialstate","numsteps","maxsteps","flux thres","cov thres","max tries","thermalisation"]
        data = [params,outcomes,is,numsteps,maxsteps,fluxt,covt,tries,ntherm]
        writedlm(io,reshape(titles,1,length(titles)))
        writedlm(io,reshape(data,1,length(data)))
    end;
    open(folder*"paramset_$(i)_simresults.tsv", "w") do io
        titles = ["sim #","Perturbed param","Pert Size","time","steps","flux thres","cov thres","<x>","<y>","Var(x)","Var(y)","cov(x,y)","usedparams"]
        writedlm(io, reshape(titles,1,length(titles)))
    end;
    for j = 1:nsims
        statevector = copy(is)
        pertsize = rand(perts)
        pertparams[2*(j%2)+1] *= pertsize
        rng = Random.Xoshiro()
        calcstor,totalsteps,trynum,fluxcheck,cbvals = reach_balance_test(statevector,pertparams,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)
        open(folder*"paramset_$(i)_simresults.tsv", "a+") do io
            data = [j,2*(j%2)+1,pertsize,calcstor.weightsum,totalsteps,fluxcheck,cbvals,calcstor.meanx,calcstor.meany,calcstor.VX/calcstor.weightsum,calcstor.VY/calcstor.weightsum,calcstor.CXY/calcstor.weightsum,pertparams]
            writedlm(io, reshape(data,1,length(data)))
        end;
        pertparams[2*(j%2)+1] /= pertsize
    end
end

In [5]:
repeat([[50.,10.,5.,1.]],10)

10-element Vector{Vector{Float64}}:
 [50.0, 10.0, 5.0, 1.0]
 [50.0, 10.0, 5.0, 1.0]
 [50.0, 10.0, 5.0, 1.0]
 [50.0, 10.0, 5.0, 1.0]
 [50.0, 10.0, 5.0, 1.0]
 [50.0, 10.0, 5.0, 1.0]
 [50.0, 10.0, 5.0, 1.0]
 [50.0, 10.0, 5.0, 1.0]
 [50.0, 10.0, 5.0, 1.0]
 [50.0, 10.0, 5.0, 1.0]

Being very careful to have fully independent perts: this should work even for low sim number

In [8]:
folder = "no_fb_sims_indtest_tauslarge_"*Dates.format(today(),"yyyy-mm-dd")*"/"
mkdir(folder)
nsims = 16 
is = [10,10]
perts = repeat([[1.9,1.,1.,1.],[1.,0.1,1.,1.],[1.,1.,1.9,1.],[1.,1.,1.,0.1],[0.1,1.,1.,1.],[1.,1.9,1.,1],[1.,1.,0.1,1.],[1.,1.,1.,1.9]],2)
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e8)
fluxt = 0.02
covt = 0.05
tries = 40
ntherm = Int(10e5)

paramsets = [[25.,13.,5.,2.],
    [40.,1.,40.,1.],
    [25.,1.,2.,1.]]
for (k,params) in enumerate(paramsets)
    i = k
    println("Paramset $(paramsets)")
    pertparams = copy(params)
    open(folder*"paramset_$(i)_settings.tsv", "w") do io
        titles = ["params","outcomes","initialstate","numsteps","maxsteps","flux thres","cov thres","max tries","thermalisation"]
        data = [params,outcomes,is,numsteps,maxsteps,fluxt,covt,tries,ntherm]
        writedlm(io,reshape(titles,1,length(titles)))
        writedlm(io,reshape(data,1,length(data)))
    end;
    open(folder*"paramset_$(i)_simresults.tsv", "w") do io
        titles = ["sim #","Perturbed param","Pert Size","time","steps","flux thres","cov thres","<x>","<y>","Var(x)","Var(y)","cov(x,y)","usedparams"]
        writedlm(io, reshape(titles,1,length(titles)))
    end;
    for (j,perturbation) in enumerate(perts)
        statevector = copy(is)
        pertparams .*= perturbation
        rng = Random.Xoshiro()
        calcstor,totalsteps,trynum,fluxcheck,cbvals = reach_balance_test(statevector,pertparams,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)
        open(folder*"paramset_$(i)_simresults.tsv", "a+") do io
            data = [0,0,0,calcstor.weightsum,totalsteps,fluxcheck,cbvals,calcstor.meanx,calcstor.meany,calcstor.VX/calcstor.weightsum,calcstor.VY/calcstor.weightsum,calcstor.CXY/calcstor.weightsum,pertparams]
            writedlm(io, reshape(data,1,length(data)))
        end;
        pertparams ./= perturbation
    end
end

Paramset [[25.0, 13.0, 5.0, 2.0], [40.0, 1.0, 40.0, 1.0], [25.0, 1.0, 2.0, 1.0]]
Paramset [[25.0, 13.0, 5.0, 2.0], [40.0, 1.0, 40.0, 1.0], [25.0, 1.0, 2.0, 1.0]]
Paramset [[25.0, 13.0, 5.0, 2.0], [40.0, 1.0, 40.0, 1.0], [25.0, 1.0, 2.0, 1.0]]


Using taus, but random perts

In [10]:
folder = "no_fb_sims_indtest_tausrandommany_"*Dates.format(today(),"yyyy-mm-dd")*"/"
mkdir(folder)
nsims = 100 
is = [10,10]
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e8)
fluxt = 0.02
covt = 0.05
tries = 40
ntherm = Int(10e5)

paramsets = [[25.,5.,5.,2.],
    [40.,5.,40.,1.],
    [25.,1.,2.,1.]]
for (k,params) in enumerate(paramsets)
    i = k
    println("Paramset $(paramsets)")
    pertparams = copy(params)
    open(folder*"paramset_$(i)_settings.tsv", "w") do io
        titles = ["params","outcomes","initialstate","numsteps","maxsteps","flux thres","cov thres","max tries","thermalisation"]
        data = [params,outcomes,is,numsteps,maxsteps,fluxt,covt,tries,ntherm]
        writedlm(io,reshape(titles,1,length(titles)))
        writedlm(io,reshape(data,1,length(data)))
    end;
    open(folder*"paramset_$(i)_simresults.tsv", "w") do io
        titles = ["sim #","Perturbed param","Pert Size","time","steps","flux thres","cov thres","<x>","<y>","Var(x)","Var(y)","cov(x,y)","usedparams"]
        writedlm(io, reshape(titles,1,length(titles)))
    end;
    for j in 1:nsims
        statevector = copy(is)
        pertchoice = rand([1,2,3,4])
        pertsize = rand(Uniform(0.8,1.2))
        pertparams[pertchoice] *= pertsize
        rng = Random.Xoshiro()
        calcstor,totalsteps,trynum,fluxcheck,cbvals = reach_balance_test(statevector,pertparams,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)
        open(folder*"paramset_$(i)_simresults.tsv", "a+") do io
            data = [j,pertchoice,pertsize,calcstor.weightsum,totalsteps,fluxcheck,cbvals,calcstor.meanx,calcstor.meany,calcstor.VX/calcstor.weightsum,calcstor.VY/calcstor.weightsum,calcstor.CXY/calcstor.weightsum,pertparams]
            writedlm(io, reshape(data,1,length(data)))
        end;
        pertparams[pertchoice] /= pertsize
    end
end

Paramset [[25.0, 5.0, 5.0, 2.0], [40.0, 5.0, 40.0, 1.0], [25.0, 1.0, 2.0, 1.0]]
Paramset [[25.0, 5.0, 5.0, 2.0], [40.0, 5.0, 40.0, 1.0], [25.0, 1.0, 2.0, 1.0]]
Paramset [[25.0, 5.0, 5.0, 2.0], [40.0, 5.0, 40.0, 1.0], [25.0, 1.0, 2.0, 1.0]]


In [20]:
a = rand([1,2,3,4])

4

0.7573762088681422
0.8038151073929121
1.8404951000514016
1.8104978863514471
0.09759942440996139
0.18966802976405694
1.8205324377823078
0.8973307484994535
-0.22173869165270155


Generate truly independent perts: cartesian product of all the values you want to multiply

In [4]:
lambdas = [1,1.4,0.9]
betaxs = [1,1.9,0.5]
gammas = [1,1.1,0.9]
betays = [1]

independentperts = [[d_l,d_bx,d_g,d_by] for d_l in lambdas for d_bx in betaxs for d_g in gammas for d_by in betays]

27-element Vector{Vector{Float64}}:
 [1.0, 1.0, 1.0, 1.0]
 [1.0, 1.0, 1.1, 1.0]
 [1.0, 1.0, 0.9, 1.0]
 [1.0, 1.9, 1.0, 1.0]
 [1.0, 1.9, 1.1, 1.0]
 [1.0, 1.9, 0.9, 1.0]
 [1.0, 0.5, 1.0, 1.0]
 [1.0, 0.5, 1.1, 1.0]
 [1.0, 0.5, 0.9, 1.0]
 [1.4, 1.0, 1.0, 1.0]
 [1.4, 1.0, 1.1, 1.0]
 [1.4, 1.0, 0.9, 1.0]
 [1.4, 1.9, 1.0, 1.0]
 ⋮
 [1.4, 0.5, 1.0, 1.0]
 [1.4, 0.5, 1.1, 1.0]
 [1.4, 0.5, 0.9, 1.0]
 [0.9, 1.0, 1.0, 1.0]
 [0.9, 1.0, 1.1, 1.0]
 [0.9, 1.0, 0.9, 1.0]
 [0.9, 1.9, 1.0, 1.0]
 [0.9, 1.9, 1.1, 1.0]
 [0.9, 1.9, 0.9, 1.0]
 [0.9, 0.5, 1.0, 1.0]
 [0.9, 0.5, 1.1, 1.0]
 [0.9, 0.5, 0.9, 1.0]

In [5]:
folder = "no_fb_sims_indtest_betasind_"*Dates.format(today(),"yyyy-mm-dd")*"/"
mkdir(folder)
nsims = 16 
is = [10,10]
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e8)
fluxt = 0.02
covt = 0.05
tries = 40
ntherm = Int(10e5)

paramsets = [[25.,3.,5.,2.],
    [40.,1.,40.,1.],
    [25.,1.,2.,1.]]
for (k,params) in enumerate(paramsets)
    i = k
    println("Paramset $(paramsets)")
    pertparams = copy(params)
    open(folder*"paramset_$(i)_settings.tsv", "w") do io
        titles = ["params","outcomes","initialstate","numsteps","maxsteps","flux thres","cov thres","max tries","thermalisation"]
        data = [params,outcomes,is,numsteps,maxsteps,fluxt,covt,tries,ntherm]
        writedlm(io,reshape(titles,1,length(titles)))
        writedlm(io,reshape(data,1,length(data)))
    end;
    open(folder*"paramset_$(i)_simresults.tsv", "w") do io
        titles = ["sim #","Perturbed param","Pert Size","time","steps","flux thres","cov thres","<x>","<y>","Var(x)","Var(y)","cov(x,y)","usedparams"]
        writedlm(io, reshape(titles,1,length(titles)))
    end;
    for (j,perturbation) in enumerate(independentperts)
        statevector = copy(is)
        pertparams .*= perturbation
        rng = Random.Xoshiro()
        calcstor,totalsteps,trynum,fluxcheck,cbvals = reach_balance_test(statevector,pertparams,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)
        open(folder*"paramset_$(i)_simresults.tsv", "a+") do io
            data = [0,0,0,calcstor.weightsum,totalsteps,fluxcheck,cbvals,calcstor.meanx,calcstor.meany,calcstor.VX/calcstor.weightsum,calcstor.VY/calcstor.weightsum,calcstor.CXY/calcstor.weightsum,pertparams]
            writedlm(io, reshape(data,1,length(data)))
        end;
        pertparams ./= perturbation
    end
end

Paramset [[25.0, 3.0, 5.0, 2.0], [40.0, 1.0, 40.0, 1.0], [25.0, 1.0, 2.0, 1.0]]
Paramset [[25.0, 3.0, 5.0, 2.0], [40.0, 1.0, 40.0, 1.0], [25.0, 1.0, 2.0, 1.0]]
Paramset [[25.0, 3.0, 5.0, 2.0], [40.0, 1.0, 40.0, 1.0], [25.0, 1.0, 2.0, 1.0]]
